<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [2]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [3]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2', 'is_duplicate'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

In [4]:
df = pd.read_csv("train.csv")
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [5]:
print(df1.columns)
print(df2.columns)
print(df.columns)

Index(['id', 'cwc_min', 'cwc_max', 'csc_min', 'csc_max', 'ctc_min', 'ctc_max',
       'last_word_eq', 'first_word_eq', 'abs_len_diff', 'mean_len',
       'token_set_ratio', 'token_sort_ratio', 'fuzz_ratio',
       'fuzz_partial_ratio', 'longest_substr_ratio'],
      dtype='object')
Index(['id', 'freq_qid1', 'freq_qid2', 'q1len', 'q2len', 'q1_n_words',
       'q2_n_words', 'word_Common', 'word_Total', 'word_share', 'freq_q1+q2',
       'freq_q1-q2'],
      dtype='object')
Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')


In [6]:
# df1['id'] = df2['id']
# aaa.reset_index(drop=True, inplace=True)
# df2['id'] = df['id']
temp_df = df1.merge(df2, on = 'id', how = 'left')
final_df = temp_df.merge(df, on = 'id', how = 'left')

In [7]:
final_df

,id,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,...,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,qid1,qid2,question1,question2,is_duplicate
0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,...,10.0,23.0,0.434783,2,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,...,4.0,20.0,0.200000,5,3,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,...,4.0,24.0,0.166667,2,0,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,...,0.0,19.0,0.000000,2,0,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,...,2.0,20.0,0.100000,4,2,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,0.666656,0.571420,0.888879,0.799992,0.705878,0.705878,1.0,0.0,0.0,...,8.0,31.0,0.258065,2,0,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,7.0,...,0.0,14.0,0.000000,2,0,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,0.499975,0.499975,0.599988,0.428565,0.571420,0.444440,1.0,0.0,2.0,...,4.0,16.0,0.250000,2,0,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,0.999950,0.499988,0.999975,0.799984,0.857131,0.749991,0.0,1.0,1.0,...,6.0,16.0,0.375000,3,1,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,0.599988,0.499992,0.333322,0.249994,0.444440,0.444440,0.0,0.0,0.0,...,3.0,18.0,0.166667,2,0,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [8]:
y_true = df.is_duplicate

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test, Y_train, Y_test = train_test_split(final_df, y_true, stratify=y_true, test_size=0.3)

In [10]:
print(X_train.shape)
print(X_test.shape)
# print(283003/(283003+121287)*100)

(283003, 32)
(121287, 32)


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(X_train['question1']) + list(X_train['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [12]:
nlp = spacy.load('en_core_web_sm')

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [13]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_train['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
X_train['q1_feats_m'] = list(vecs1)


100%|█████████████████████████████████████████████████████████████████████████| 283003/283003 [40:18<00:00, 116.99it/s]


In [14]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_test['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
X_test['q1_feats_m'] = list(vecs1)


100%|█████████████████████████████████████████████████████████████████████████| 121287/121287 [17:38<00:00, 114.56it/s]


In [15]:
vecs2 = []
for qu2 in tqdm(list(X_train['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
X_train['q2_feats_m'] = list(vecs2)

100%|█████████████████████████████████████████████████████████████████████████| 283003/283003 [43:12<00:00, 109.15it/s]


In [16]:
vecs2 = []
for qu2 in tqdm(list(X_test['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
X_test['q2_feats_m'] = list(vecs2)

100%|█████████████████████████████████████████████████████████████████████████| 121287/121287 [16:47<00:00, 120.33it/s]


TypeError: 'numpy.ndarray' object is not callable

In [17]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [16]:
# df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
# df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
# # df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

# # df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
# # df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [17]:
# X_train_ = X_train.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
# X_test_ = X_test.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

In [26]:
# df3_q1_train = pd.DataFrame(X_train_.q1_feats_m.values.tolist(), index= X_train.index)
# df3_q2_train = pd.DataFrame(X_train_.q2_feats_m.values.tolist(), index= X_train.index)
train_q1 = pd.DataFrame(X_train.q1_feats_m.values.tolist(), index= X_train.index)
train_q2 = pd.DataFrame(X_train.q2_feats_m.values.tolist(), index= X_train.index)
test_q1 = pd.DataFrame(X_test.q1_feats_m.values.tolist(), index= X_test.index)
test_q2 = pd.DataFrame(X_test.q2_feats_m.values.tolist(), index= X_test.index)


In [32]:
# train_q1['id'] = X_train['id']
# train_q2['id'] = X_train['id']
# train_q = train_q1.merge(train_q2, on = 'id', how = 'left')
# result_train = X_train.merge(train_q, on = 'id', how = 'left')
# # storing the final features to csv file
# # if not os.path.isfile('final_features.csv'):
# #     result_train.to_csv('final_features_train.csv')

In [35]:
test_q1['id'] = X_test['id']
test_q2['id'] = X_test['id']
test_q = test_q1.merge(test_q2, on = 'id', how = 'left')
result_test = X_test.merge(test_q, on = 'id', how = 'left')
# storing the final features to csv file
# if not os.path.isfile('final_features.csv'):
    result_test.to_csv('final_features_test.csv')

In [36]:
print(result_train.shape)
print(result_test.shape)

(283003, 802)
(121287, 802)


In [40]:
import pickle
with open('quora_variables', 'wb') as f:
    pickle.dump((result_train, result_test), f)

In [20]:
df3_q1_test = pd.DataFrame(X_test.q1_feats_m.values.tolist(), index= X_test.index)
df3_q2_test = pd.DataFrame(X_test.q2_feats_m.values.tolist(), index= X_test.index)

In [42]:
# df3_q1_test

In [21]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [22]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [24]:
# Questions 1 tfidf weighted word2vec
df3_q1_train.head()

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
76476,44.141706,40.112920,48.034789,82.955543,-24.813502,43.979569,11.992027,-36.466090,17.616706,54.310567,...,10.047320,8.508925,8.184625,-2.223166,4.946005,10.254384,-10.626677,1.302959,4.305164,0.709454
203893,-8.867132,7.973247,23.498013,47.612614,-34.197906,10.502384,-61.377902,-49.033760,23.147168,12.793280,...,5.486010,6.102271,-9.906230,-4.070686,10.415692,-3.876022,-5.302826,5.900097,2.072360,-0.460279
293637,69.631675,-25.760675,-39.057670,115.206744,-15.623750,-0.971421,-179.273942,-70.480685,43.111605,50.521561,...,27.912493,33.301450,15.410098,2.063384,23.436080,-4.734484,-3.018346,-8.707124,16.974567,-3.353133
134819,0.460998,5.281210,8.067459,142.219796,-50.527783,-29.815671,-84.533616,-39.835542,-13.112048,-43.862353,...,5.056737,19.815207,12.102962,-11.796170,13.716933,6.296513,-8.721851,6.354566,21.662184,6.648943
247866,12.159539,-9.054222,-2.819261,5.401913,-64.197112,58.270283,-75.788241,-8.936225,72.202427,49.563517,...,-3.667833,14.268259,3.659748,-4.555014,6.466452,-3.553775,-13.938177,5.543483,7.597044,-4.178000


In [27]:
# Questions 2 tfidf weighted word2vec
df3_q2_train.head()

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
76476,39.307488,38.865929,55.534151,58.905942,-21.861104,33.119402,35.970366,-28.145900,26.831802,23.857513,...,12.996673,0.624764,6.933407,-1.781185,7.885681,10.424353,-7.584718,3.645061,6.940381,0.358635
203893,1.197543,26.019416,14.950367,66.369663,-24.925778,6.335563,-54.965989,-48.421855,20.341785,13.328071,...,1.976444,5.393134,-6.964031,-2.018588,12.161211,0.001232,-5.622833,0.807230,3.438480,2.087152
293637,50.701604,23.033240,66.647519,98.993092,35.595808,-19.480462,-82.108417,-96.858718,-2.887819,46.825997,...,27.450563,38.316267,8.894856,-9.269991,28.694674,-3.707579,-20.067194,9.555296,9.986240,13.159571
134819,34.013277,35.525395,33.767069,129.215463,-48.433789,44.768901,-79.929721,-48.354844,72.086764,16.420888,...,-5.505090,16.222737,7.200384,-15.098252,17.916683,7.003365,-19.710984,15.628867,-2.847528,0.357586
247866,60.451401,42.484428,-22.248498,107.052141,-16.145114,73.964728,-232.572158,-2.526386,172.330258,-41.399531,...,28.144895,14.657734,11.823552,0.861218,-1.874851,2.330577,-28.561426,23.116660,9.608572,-10.455854


In [28]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print('#'*30,'Train','#'*30)
print("Number of features in question1 w2v  dataframe Train :", df3_q1_train.shape[1])
print("Number of features in question2 w2v  dataframe Train :", df3_q2_train.shape[1])
print("Number of features in final dataframe Train :", df1.shape[1]+df2.shape[1]+df3_q1_train.shape[1]+df3_q2_train.shape[1])

print('#'*30,'Test','#'*30)

print("Number of features in question1 w2v  dataframe test :", df3_q1_test.shape[1])
print("Number of features in question2 w2v  dataframe test :", df3_q2_test.shape[1])
print("Number of features in final dataframe test :", df1.shape[1]+df2.shape[1]+df3_q1_test.shape[1]+df3_q2_test.shape[1])


Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
############################## Train ##############################
Number of features in question1 w2v  dataframe Train : 384
Number of features in question2 w2v  dataframe Train : 384
Number of features in final dataframe Train : 797
############################## Test ##############################
Number of features in question1 w2v  dataframe test : 384
Number of features in question2 w2v  dataframe test : 384
Number of features in final dataframe test : 797


,id,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,...,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,...,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,...,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,...,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,...,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,...,1,76,39,13,7,2.0,20.0,0.100000,4,2
5,5,0.666656,0.571420,0.888879,0.799992,0.705878,0.705878,1.0,0.0,0.0,...,1,86,90,16,16,8.0,31.0,0.258065,2,0
6,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,7.0,...,1,19,62,4,11,0.0,14.0,0.000000,2,0
7,7,0.499975,0.499975,0.599988,0.428565,0.571420,0.444440,1.0,0.0,2.0,...,1,30,41,7,9,4.0,16.0,0.250000,2,0
8,8,0.999950,0.499988,0.999975,0.799984,0.857131,0.749991,0.0,1.0,1.0,...,2,31,37,8,8,6.0,16.0,0.375000,3,1
9,9,0.599988,0.499992,0.333322,0.249994,0.444440,0.444440,0.0,0.0,0.0,...,1,60,49,9,9,3.0,18.0,0.166667,2,0


In [ ]:
df3_q1_train['id']=df1_train['id']
df3_q2_train['id']=df1_train['id']
# df3_q1_test['id']=df1_train['id']
# df3_q2_test['id']=df1_train['id']
df1_train  = df1_train.merge(df2_train, on='id',how='left')
df2_train  = df3_q1_train.merge(df3_q2_train, on='id',how='left')
result_train = df1_train.merge(df2_train, on='id',how='left')

In [ ]:
df3_q1_test['id']=df1_test['id']
df3_q2_test['id']=df1_test['id']
# df3_q1_test['id']=df1_test['id']
# df3_q2_test['id']=df1_test['id']
df1_test  = df1_test.merge(df2_test, on='id',how='left')
df2_test  = df3_q1_test.merge(df3_q2_test, on='id',how='left')
result_test = df1_test.merge(df2_test, on='id',how='left')

In [ ]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1_train['id']=df1['id']
    df3_q2_train['id']=df1['id']
    df3_q1_test['id']=df1['id']
    df3_q2_test['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features.csv')

In [30]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1_train['id']=df1_train['id']
    df3_q2_train['id']=df1_train['id']
#     df3_q1_test['id']=df1_train['id']
#     df3_q2_test['id']=df1_train['id']
    df1_train  = df1_train.merge(df2_train, on='id',how='left')
    df2_train  = df3_q1_train.merge(df3_q2_train, on='id',how='left')
    result_train  = df1_train.merge(df2_train, on='id',how='left')
    result_train.to_csv('final_features_train.csv')

In [31]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
#     df3_q1_train['id']=df1_train['id']
#     df3_q2_train['id']=df1_train['id']
    df3_q1_test['id']=df1_train['id']
    df3_q2_test['id']=df1_train['id']
    df1_test  = df1_train.merge(df2_train, on='id',how='left')
    df2_test  = df3_q1_test.merge(df3_q2_train, on='id',how='left')
    result_test  = df1_test.merge(df2_train, on='id',how='left')
    result_test.to_csv('final_features_test.csv')